In [1]:
#!pip install git+https://github.com/openai/swarm.git

In [2]:
import os
with open("secret/openai_api_key.txt", "r") as f:
    OPENAI_API_KEY = f.read()
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
from swarm import Swarm, Agent
from swarm.types import Result

![diagram](./assets/diagram.png)

# `context_variables`

In [4]:
context_variables = {"user_name": None,
                     "cart": []}

# Agents

## Receptionist

In [5]:
def carting(item: str):
    context_variables['cart'] += [item]
    return Result(value="cart updated.",
                  context_variables=context_variables, 
                  )
    # Use Result to update context_variables after running a tool.
    

def payment():
    return "payment processed."

def update_user_name(name: str):
    context_variables['user_name'] = name
    context_variables['update_crm_needed'] = True
    return Result(value=f"Hi {name}", agent=agent_receptionist)

# Agent's instruction can be a callable that returns a string.
def inst_reception_agent(context_variables):
    output = "You are a helpful receptionist at a cafe."
    "You will be friendlily ahd happily help a customer buy products from the cafe."
    "The cafe only sells specialty coffee beans and specialty tea leaves. We do not sell drinks. "
    "Therefore, when customers just mention coffee, it means coffee beans, and you politely clarify to the customer. "
    "When customers just mention tea, it means tea leaves, and you politely clarify to the customers. "
    "You can do small talk, but circle back to ask if you can help the customer choose products from our cafe. "
    "You are also responsible for the carting system and the payment system. "
    "When asked about what products we sell, ask the experts (both cofee expert AI and tea expert AI) of what products are available. "
    "When you are asked about the detail of the products, you will ask other AI agents who are specialized to know "
    "more about the products."
    if context_variables['user_name']:
        output += f"Today, you will serve a customer named {context_variables['user_name']}"
    return output

agent_receptionist = Agent(
    name="Receptionist",
    instructions=inst_reception_agent,
    functions=[carting, payment, update_user_name],
)

## Coffee Expert


In [6]:
with open("assets/coffee-data.txt", "r") as f:
    COFFEE_DATA = f.read()

def coffee_search():
    return Result(value=COFFEE_DATA, agent=agent_receptionist)
    # Also use Result to specify the next agent. It is better for this application.

agent_coffee = Agent(
    name="Coffee Expert",
    instructions="You are expert in specialty coffee beans. You will answer coffee-related questions. "
    "You must use `coffee_search` tool to retrieve proprietary information, and use this "
    "piece of information to answer first. If the information here is not relevant to the question, "
    "you may attempt to answer with your own knowledge, but you must admit that you do not have backup information."
    "If the users ask about anything unreleated to coffee products, you handoff to the receptionist AI. ",
    functions=[coffee_search]
)

## Tea Expert

In [7]:
with open("assets/tea-data.txt","r") as f:
    TEA_DATA = f.read()
    
def tea_search():
    return TEA_DATA
    # This tools does not specify the next agent. So, there is a chance that the flow gets messed up.

agent_tea = Agent(
    name="Tea Expert",
    instructions="You are expert in specialty tea leaves. You will answer tea-related questions. "
    "You must use `tea_search` tool to retrieve proprietary information, and use this "
    "piece of information to answer first. If the information here is not relevant to the question, "
    "you may attempt to answer with your own knowledge, but you must admit that you do not have backup information."
    "If the users ask about anything unreleated to tea products, you handoff to the receptionist AI. ",
    functions=[tea_search]
)

## Connections

In [8]:
def transfer_to_receptionist():
    return agent_receptionist

def transfer_to_coffee_expert():
    return agent_coffee

def transfer_to_tea_expert():
    return agent_tea

In [9]:
agent_receptionist.functions.extend([transfer_to_coffee_expert, transfer_to_tea_expert])
agent_coffee.functions.append(transfer_to_receptionist)
agent_tea.functions.append(transfer_to_receptionist)

# Run

In [10]:
client = Swarm()

In [11]:
messages = [{"role":"user", "content": "what coffee bean do you have?"}]

In [12]:
responses = client.run(
    agent=agent_receptionist,
    messages=messages,
    context_variables=context_variables,
    max_turns=float('inf'),
    model_override="gpt-4o-mini",
    execute_tools=True, #If False, interrupt execution and immediately returns tool_calls message when an Agent tries to call a function
    stream=False,
    debug=False
)

In [13]:
responses.__dict__

{'messages': [{'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_h2L5rkJ38IyxXyTmrt9o96pA',
     'function': {'arguments': '{}', 'name': 'transfer_to_coffee_expert'},
     'type': 'function'}],
   'sender': 'Receptionist'},
  {'role': 'tool',
   'tool_call_id': 'call_h2L5rkJ38IyxXyTmrt9o96pA',
   'tool_name': 'transfer_to_coffee_expert',
   'content': '{"assistant": "Coffee Expert"}'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_zAo6kOQ2Xpypbl8RP1sfWnMY',
     'function': {'arguments': '{}', 'name': 'coffee_search'},
     'type': 'function'}],
   'sender': 'Coffee Expert'},
  {'role': 'tool',
   'tool_call_id': 'call_zAo6kOQ2Xpypbl8RP1sfWnMY',
   'tool_name': 'coffee_search',
   'content': "# Product Master of Specialty Coffee Beans\n\n## 1. Ethiopian Yirgacheffe\n- **Origin**: Yirgacheffe, Ethiopia, a regio

# `run_demo_loop`

In [14]:
from swarm.repl import run_demo_loop

In [17]:
run_demo_loop(agent_receptionist)
# when running this, chat history is cleared every time.

Starting Swarm CLI 🐝


Receptionist: transfer_to_coffee_expert()
Coffee Expert: coffee_search()
Receptionist: We offer three specialty coffee beans:

1. **Ethiopian Yirgacheffe**  
   - **Taste**: Bright acidity with floral and fruity notes like jasmine and lemon.
   - **Recommended Brewing**: Pour-over (e.g., Chemex or V60).
   - **Price per 250g**: $18.

2. **Jamaican Blue Mountain**  
   - **Taste**: Smooth and mild, with balanced acidity and notes of chocolate and nuts.
   - **Recommended Brewing**: French Press or siphon.
   - **Price per 250g**: $45.

3. **Panama Geisha**  
   - **Taste**: Complex and aromatic with flavors of jasmine, peach, and tropical fruits.
   - **Recommended Brewing**: Pour-over (e.g., Kalita Wave).
   - **Price per 250g**: $50.

If you're interested in trying any of these, let me know!
Receptionist: carting("item"= "Jamaican Blue Mountain")
Receptionist: The Jamaican Blue Mountain coffee has been added to your cart. If you wish to proceed with the purchase or need further assist

KeyboardInterrupt: Interrupted by user

In [18]:
context_variables

{'user_name': 'Benz',
 'cart': ['Darjeeling First Flush Black Tea',
  'Jamaican Blue Mountain',
  'Gyokuro Japanese Green Tea'],
 'update_crm_needed': True}